In [1]:
import os
import boto3
import argparse
import sagemaker
from datetime import datetime
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements
from sagemaker.predictor import Predictor
from sagemaker.enums import EndpointType
from sagemaker.model import Model
from sagemaker.session import Session
from sagemaker.xgboost import XGBoostModel
from sagemaker.pipeline import PipelineModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/mak/.config/sagemaker/config.yaml


In [2]:
model_name = "xgboost"
endpoint_name = "xgboost"
endpoint_config_name = "xgboost"
role = "arn:aws:iam::767397884728:role/SageMakerSnowFlakeExampleIAMRole-"
# boto3.setup_default_session(profile_name='ml-staging-admin')
region = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = region
session = boto3.session.Session(profile_name="ml-staging-admin")
sagemaker_session = sagemaker.Session(session)


In [3]:
model_data = "s3://sagemaker-us-east-1-767397884728/pipelines-zfpv1p4n1mci-XgBoostModelTraining-sd99CPgt5T/output/model.tar.gz"
xgboost_model = XGBoostModel(
    model_data=model_data,
    role=role,
    framework_version='1.7-1',
    source_dir="code",
    entry_point='xgboost_inference.py',
    sagemaker_session=sagemaker_session,
    name=model_name,
)

In [4]:
pipeline_model_name = "xgboostpipeline"
pipeline_model = PipelineModel(models=[xgboost_model],role=role,sagemaker_session=sagemaker_session,
                                name=pipeline_model_name)
pipeline_model.deploy(instance_type='ml.m4.xlarge',
        initial_instance_count=1,
        endpoint_name=endpoint_name)

Using already existing model: xgboostpipeline


-----------------!